In [ ]:
import os
import cv2
import sys
import timm
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F

from torch import nn
from tqdm import tqdm
from skimage import io
from pathlib import Path
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
# serve per ricaricare automaticamente il codice modificato
%load_ext autoreload
%autoreload 2

In [ ]:
# directory principale di training, perche' nel file .csv ci sono solo i nomi dei file (immagini), da modificare per 
path = Path(os.getcwd()).parent.parent
real_data_dir = os.path.join(path, "artifact", "coco")
fake_data_dir = os.path.join(path, "artifact", "latent_diffusion")

BATCH_SIZE = 32

LR = 0.001

EPOCHS = 30

DEVICE = 'cuda'

In [ ]:
# df che devo creare
csv_path = os.path.join("..", "datasets", "out.csv")
df = pd.read_csv(csv_path)
#df.head()
df = df.sample(frac=1)
train_df, valid_df = train_test_split(df, test_size=0.20, random_state=42)

print(train_df.size)
print(valid_df.size)

In [ ]:
# carica le immagini nel dataset
class APN_Dataset(Dataset):

  def __init__(self, df):
    self.df = df

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    row = self.df.iloc[idx]
    
    #2   0_img = io.imread(os.path.join(real_data_dir, row.Anchor))
    #P_img = io.imread(os.path.join(real_data_dir, row.Positive))
    #N_img = io.imread(os.path.join(fake_data_dir, row.Negative))
    
    
    if "coco" in row.Anchor:
        A_img = io.imread(os.path.join(real_data_dir, row.Anchor))
        P_img = io.imread(os.path.join(real_data_dir, row.Positive))
        N_img = io.imread(os.path.join(fake_data_dir, row.Negative))
    
    else:
        A_img = io.imread(os.path.join(fake_data_dir, row.Anchor))
        P_img = io.imread(os.path.join(fake_data_dir, row.Positive))
        N_img = io.imread(os.path.join(real_data_dir, row.Negative))

    # permute because the third channel has to be in first channel in torch

    A_img = torch.from_numpy(A_img).permute(2, 0, 1) / 255.0
    P_img = torch.from_numpy(P_img).permute(2, 0, 1) / 255.0
    N_img = torch.from_numpy(N_img).permute(2, 0, 1) / 255.0
    
    #A_img = np.expand_dims(A_img, 0)
    #P_img = np.expand_dims(P_img, 0)
    #N_img = np.expand_dims(N_img, 0)
    
    # normalizzazione per non far divergere il comportamento della rete
    # il valore dell'immagine sarà compreso tra 0 e 1
    #A_img = torch.from_numpy(A_img) / 255.0
    #P_img = torch.from_numpy(P_img) / 255.0
    #N_img = torch.from_numpy(N_img) / 255.0

    #A_img = torch.from_numpy(A_img.astype(np.int32)) / 65536.0
    #P_img = torch.from_numpy(P_img.astype(np.int32)) / 65536.0
    #N_img = torch.from_numpy(N_img.astype(np.int32)) / 65536.0

    return A_img, P_img, N_img

In [ ]:
trainset = APN_Dataset(train_df)
validset = APN_Dataset(valid_df)

print(f"Size of trainset: {len(trainset)}")
print(f"Size of validset: {len(validset)}")

In [ ]:
trainloader = DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True)
validloader = DataLoader(validset, batch_size = BATCH_SIZE)

In [ ]:
print(f"No. of batches in trainloader : {len(trainloader)}")
print(f"No. of batches in validloader : {len(validloader)}")

In [ ]:
# FUNZIONE PER CARICARE IL MODELLO DI RETE NEURALE DIRETTAMENTE DALLE REPOSITORY ONLINE
class APN_Model(nn.Module):

  # QUI DEFINISCO LA SIZE DEL VETTORE DI EMBEDDING
  def __init__(self, emb_size = 512):
    super(APN_Model, self).__init__()

    # QUI CAIRCATE IL MODELLO, IN QUESTO CASO EFFICIENTNET VERSIONE B0 (LA PIù LEGGERA DELLA FAMIGLIA)
    self.efficientnet = timm.create_model('tf_efficientnetv2_b0', pretrained=False)
    self.efficientnet.classifier = nn.Linear(in_features=self.efficientnet.classifier.in_features, out_features=emb_size)

  def forward(self, images):
    embeddings = self.efficientnet(images)
    return embeddings

In [ ]:
# QUI FATE UNA PICCOLA MODIFICA ALLA RETE PER FARLE AVERE IN INPUT IMMAGINI IN SCALA DI GRIGIO DELLO SPETTRO DI FOURIER
model = APN_Model()
#model.efficientnet.conv_stem = nn.Conv2d(1, 32, 3, 2, 1, bias=False)

model.to(DEVICE)

In [ ]:
# funzione di train
def train_fn(model, dataloader, optimizer, criterion):
  model.train() #ON Dropout
  total_loss = 0.0

  for A, P, N in tqdm(dataloader):
    A, P, N = A.to(DEVICE), P.to(DEVICE), N.to(DEVICE)

    print("debug, A shape: " + str(A.shape), file=sys.stderr)
    print("debug, P shape: " + str(P.shape), file=sys.stderr)
    print("debug, N shape: " + str(N.shape), file=sys.stderr)

    # qui vengono creati gli embeddings, le cui distanze verranno calcolate dopo
    A_embs = model(A)
    P_embs = model(P)
    N_embs = model(N)

    # criterion è la funzione di loss triplet
    loss = criterion(A_embs, P_embs, N_embs)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    return total_loss / len(dataloader)


In [ ]:
# funzione di evaluation
def eval_fn(model, dataloader, criterion):
  model.eval() #OFF Dropout
  total_loss = 0.0

  with torch.no_grad():
    for A, P, N in tqdm(dataloader):
      A, P, N = A.to(DEVICE), P.to(DEVICE), N.to(DEVICE)

      A_embs = model(A)
      P_embs = model(P)
      N_embs = model(N)

      loss = criterion(A_embs, P_embs, N_embs)

      total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
# triplet loss e adam
criterion = nn.TripletMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LR)

In [ ]:
# training
best_valid_loss = np.Inf

for i in range(EPOCHS):
  train_loss = train_fn(model, trainloader, optimizer, criterion)
  valid_loss = eval_fn(model, validloader, criterion)

  if valid_loss < best_valid_loss:
    torch.save(model.state_dict(), 'best_model.pt')
    best_valid_loss = valid_loss
    print("SAVED_WEIGHTS_SUCCESS")

  print(f"EPOCHS : {i+1} train_loss : {train_loss} valid_loss : {valid_loss}")

In [ ]:
# QUESTA E' LA FUNZIONE PER GENERARE I VETTORI DI ENCODING o embeddings
def get_encoding_csv(model, anc_img_names, dirFolderReal, dirFolderFake):
  anc_img_names_arr = np.array(anc_img_names)
  encodings = []

  model.eval()
  
  with torch.no_grad():
    for i in tqdm(anc_img_names_arr):
      if "coco" in i:
        A = io.imread(os.path.join(dirFolderReal,i))
        A = torch.from_numpy(A).permute(2, 0, 1) / 255.0
        #A = np.expand_dims(A, 0)
        #A = torch.from_numpy(A.astype(np.int32)) / 255.0
        A = A.to(DEVICE)
        A_enc = model(A.unsqueeze(0))
        encodings.append(A_enc.squeeze().cpu().detach().numpy())
      else:
        A = io.imread(os.path.join(dirFolderFake,i))
        A = torch.from_numpy(A).permute(2, 0, 1) / 255.0
        #A = np.expand_dims(A, 0)
        #A = torch.from_numpy(A.astype(np.int32)) / 255.0
        A = A.to(DEVICE)
        A_enc = model(A.unsqueeze(0))
        encodings.append(A_enc.squeeze().cpu().detach().numpy())         

  #with torch.no_grad():
  #  for i in tqdm(anc_img_names_arr):
  #    A = io.imread(os.path.join(dirFolderReal,i))
  #    A = torch.from_numpy(A).permute(2, 0, 1) / 255.0
  #    #A = np.expand_dims(A, 0)
  #    #A = torch.from_numpy(A.astype(np.int32)) / 255.0
  #    A = A.to(DEVICE)
  #    A_enc = model(A.unsqueeze(0))
  #    encodings.append(A_enc.squeeze().cpu().detach().numpy())

    encodings = np.array(encodings)
    encodings = pd.DataFrame(encodings)
    df_enc = pd.concat([anc_img_names, encodings], axis = 1)

    return df_enc

In [ ]:
# QUI RICARICO IL MODELLO UNA VOLTA TRAINATO
model.load_state_dict(torch.load('best_model.pt'))

# QUI CREO IL DATABASE DI FEATURE VECTORS DEL TRAINING SET
# gli embeddings vengono aggiunti nel file csv per non rifarlo ad ogni allenamento
df_enc = get_encoding_csv(model, df['Anchor'], real_data_dir, fake_data_dir)

In [ ]:
# QUI IL DATABASE COME CSV IN MODO TALE DA NON DOVER FARE QUESTA OPERAZIONE OGNI VOLTA
# OVVIAMENTE, SE DEVO FARE UN NUOVO TRAINING DEVO ANCHE RICREARE GLI ENCODINGS
df_enc.to_csv('database.csv', index = False)

df_enc = pd.read_csv('database.csv')
df_enc.head()

In [ ]:
# approssimazione della distanza, senza la radice quadrata, per fare i primi
# allenamenti velocemente
def euclidean_dist(img_enc, anc_enc_arr):
    #dist = np.sqrt(np.dot(img_enc-anc_enc_arr, (img_enc- anc_enc_arr).T))
    dist = np.dot(img_enc-anc_enc_arr, (img_enc- anc_enc_arr).T)
    #dist = np.sqrt(dist)
    return dist

In [ ]:
path = os.path.join(Path(os.getcwd()).parent, 'datasets', 'testList.csv')
df = pd.read_csv(path)

print(df['real'])
print(df.size)

df.head()

In [ ]:
def getImageEmbeddings(img, model):

    # img = np.expand_dims(img, 0)
    # img = torch.from_numpy(img) / 255
    img = torch.from_numpy(img).permute(2, 0, 1) / 255.0
    model.eval()

    with torch.no_grad():
        img = img.to(DEVICE)
        img_enc = model(img.unsqueeze(0))
        img_enc = img_enc.detach().cpu().numpy()
        img_enc = np.array(img_enc)

    return img_enc

In [ ]:
def searchInDatabase(img_enc, database):
    anc_enc_arr = database.iloc[:, 1:].to_numpy()
    anc_img_names = database['Anchor']

    distance = []
    for i in range(anc_enc_arr.shape[0]):
        dist = euclidean_dist(img_enc, anc_enc_arr[i : i+1, :])
        distance = np.append(distance, dist)

    closest_idx = np.argsort(distance)

    return database['Anchor'][closest_idx[0]]

In [ ]:
# DataTestReal = 'C:/Users/polsi/Desktop/Lavori/DeepFake/Datasets/Artifact/cycle_gan/st/test/'
path = Path(os.getcwd()).parent.parent
real_data_dir = os.path.join(path, "artifact", "coco")
fake_data_dir = os.path.join(path, "artifact", "latent_diffusion")

y_true = []
y_pred = []
tempDf = df
tempDf.head()
tempDf.shape

In [ ]:
# testo i fake
currentTest = 'fake'
database = df_enc
# prendo i primi 500 Fake
for index, row in tqdm(tempDf.iterrows()):
    path = os.path.join(fake_data_dir, row[currentTest])
    img_name = path

    img = io.imread(img_name)

    img_enc = getImageEmbeddings(img, model)

    closestLabel = searchInDatabase(img_enc, database)

    if "coco" in closestLabel:
        y_pred.append("real")
    else:
        y_pred.append("fake")

In [ ]:
print(len(y_true))
print(len(y_pred))
print(y_pred)

In [ ]:
database = df_enc

In [ ]:
# testo i real
currentTest = 'real'
# prendo i primi 500 Fake
for index, row in tqdm(tempDf.iterrows()):
    path = os.path.join(real_data_dir, row[currentTest])
    img_name = path

    img_enc = getImageEmbeddings(img, model)

    closestLabel = searchInDatabase(img_enc, database)
    if "coco" in closestLabel:
        y_pred.append("real")
    else:
        y_pred.append("fake")


In [ ]:
print(len(y_true))
print(len(y_pred))
print(y_pred)

In [ ]:
# creo i vettori di ground truth
# y_true = np.array(['fake'] * 1523)
y_true = np.array(['fake'] * 1600)
print(y_true.shape)

# temp = np.array(['real'] * 1523)
temp = np.array(['real'] * 1600)
print(temp.shape)

# y_true = np.concatenate([y_true, temp])
y_true = np.concatenate([y_true, temp])
print(y_true.shape)

# calcolo la matrice di confusione (quella di scikit-learn dispone i risultati come nella cella di sotto)
confusion_matrix(y_true, y_pred, labels=["real", "fake"])

In [ ]:
# estraggo dalla matrice di confusione i True Negative, False Positive, False Negative, True Positive
TN, FP, FN, TP = confusion_matrix(y_true, y_pred, labels=["real", "fake"]).ravel()

In [ ]:
# calcolo alcune metriche per vedere come si comporta
accuracy = round((TP + TN) /(TP + TN + FP + FN), 4) * 100
precision = round((TP) / (TP + FP), 4) * 100
sensitivy_recall = round((TP) / (TP + FN), 4) * 100
specificity = round((TN) / (TN + FP) * 100, 4)
F1_score = round((2* precision * sensitivy_recall) / (precision + sensitivy_recall), 2)

#Salvataggio del file csv
df_result = pd.DataFrame({"Precision": [precision], "Sensitivity":[sensitivy_recall], "Specificity":[specificity],"f1_score":[F1_score]})
output_dir_result = os.path.join(Path(os.getcwd()).parent, "results")
output_dir_result = os.path.join(output_dir_result, "results.csv")
df_result.to_csv(output_dir_result,index=False)

print({"Accuracy":accuracy,"Precision":precision,"Sensitivity_recall":sensitivy_recall, "Specificity": specificity, "F1_score":F1_score})